In [1]:
import random
import time
grid1 = [[]]
grid2 = [[]]
grid_size = 10
num_of_ships = 3
bullets_left = 80
game_over = False
num_of_ships_sunk_1 = 0
num_of_ships_sunk_2 = 0
ship_positions_1 = [[]]
ship_positions_2 = [[]]
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
temp1 = 0
y = [[i,j] for i in range(10) for j in range(10)]
y1 = [[i,j] for i in range(10) for j in range(10)]
hit_count = 0
hitting_axies = []
row = ''
col = ''
direction = ''
ship_acquir_empty_space = []
ship_size_list = []

def validate_grid1_and_place_ship(start_row, end_row, start_col, end_col):
    global grid1
    global ship_positions_1

    all_valid = True
    for r in range(start_row, end_row):
        for c in range(start_col, end_col):
            if grid1[r][c] != ".":
                all_valid = False
                break
    if all_valid:
        ship_positions_1.append([start_row, end_row, start_col, end_col])
        for r in range(start_row, end_row):
            for c in range(start_col, end_col):
                grid1[r][c] = "O"
    return all_valid


def try_to_place_ship_on_grid1(row, col, direction, length):
    global grid_size

    start_row, end_row, start_col, end_col = row, row + 1, col, col + 1
    if direction == "left":
        if col - length < 0:
            return False
        start_col = col - length + 1

    elif direction == "right":
        if col + length >= grid_size:
            return False
        end_col = col + length

    elif direction == "up":
        if row - length < 0:
            return False
        start_row = row - length + 1

    elif direction == "down":
        if row + length >= grid_size:
            return False
        end_row = row + length

    return validate_grid1_and_place_ship(start_row, end_row, start_col, end_col)


def print_grid1():
    global alphabet

    debug_mode = True

    alphabet = alphabet[0: len(grid1) + 1]

    for row in range(len(grid1)):
        print(alphabet[row], end=") ")
        for col in range(len(grid1[row])):
            if grid1[row][col] == "O":
                if debug_mode:
                    print("O", end=" ")
                else:
                    print(".", end=" ")
            else:
                print(grid1[row][col], end=" ")
        print("")

    print("  ", end=" ")
    for i in range(len(grid1[0])):
        print(str(i), end=" ")
    print("")


def first_player_grid():
    global grid1
    global grid_size
    global num_of_ships
    global ship_positions_1
    global ship_size_list

    random.seed(time.time())

    rows, cols = (grid_size, grid_size)

    grid1 = []
    for r in range(rows):
        row = []
        for c in range(cols):
            row.append(".")
        grid1.append(row)
    
    print_grid1()
    num_of_ships_placed = 0

    ship_positions_1 = []

    while num_of_ships_placed != num_of_ships:
        ship_size = int(input('enter_ship_size : '))
        random_row = int(input('enter_row : '))
        random_col = int(input('enter_col : '))
        direction = input('enter_direction_from_["left", "right", "up", "down"] : ')
        if try_to_place_ship_on_grid1(random_row, random_col, direction, ship_size):
            ship_size_list.append(ship_size)
            num_of_ships_placed += 1
            print_grid1()


def accept_valid_bullet_placement_1():
    global alphabet
    global grid2

    is_valid_placement = False
    row = -1
    col = -1
    while is_valid_placement is False:
        placement = input("Enter row (A-J) and column (0-9) such as A3: ")
        placement = placement.upper()
        if len(placement) <= 0 or len(placement) > 2:
            print("Error: Please enter only one row and column such as A3")
            continue
        row = placement[0]
        col = placement[1]
        if not row.isalpha() or not col.isnumeric():
            print("Error: Please enter letter (A-J) for row and (0-9) for column")
            continue
        row = alphabet.find(row)
        if not (-1 < row < grid_size):
            print("Error: Please enter letter (A-J) for row and (0-9) for column")
            continue
        col = int(col)
        if not (-1 < col < grid_size):
            print("Error: Please enter letter (A-J) for row and (0-9) for column")
            continue
        if grid2[row][col] == "#" or grid2[row][col] == "X":
            print("You have already shot a bullet here, pick somewhere else")
            continue
        if grid2[row][col] == "." or grid2[row][col] == "O":
            is_valid_placement = True

    return row, col


def check_for_ship_sunk_1(row, col):
    global ship_positions
    global grid1

    for position in ship_positions_1:
        start_row = position[0]
        end_row = position[1]
        start_col = position[2]
        end_col = position[3]
        if start_row <= row <= end_row and start_col <= col <= end_col:
            # Ship found, now check if its all sunk
            for r in range(start_row, end_row):
                for c in range(start_col, end_col):
                    if grid1[r][c] != "X":
                        return False
    return True


def player1_shoot_bullet():
    global grid2
    global num_of_ships_sunk_2
    global bullets_left
    row, col = accept_valid_bullet_placement_1()
    print("")
    print("----------------------------")
    if grid2[row][col] == ".":
        print("You missed, no ship was shot")
        grid2[row][col] = "#"
    elif grid2[row][col] == "O":
        print("You hit!", end=" ")
        grid2[row][col] = "X"
        if check_for_ship_sunk_2(row, col):
            print("A ship was completely sunk!")
            num_of_ships_sunk_2 += 1
        else:
            print("A ship was shot")
    print_grid2()
    bullets_left -= 1


def validate_grid2_and_place_ship(start_row, end_row, start_col, end_col,direction2):
    global grid
    global ship_positions
    global ship_acquir_empty_space

    all_valid = True
    for r in range(start_row, end_row):
        for c in range(start_col, end_col):
            if grid2[r][c] != ".":
                all_valid = False
                break
            if ([r,c] in ship_acquir_empty_space):
                all_valid = False
                break
    if all_valid:
        if direction2 in ['right','left']:
          ship_acquir_empty_space.extend([[start_row,start_col-1],[start_row,end_col]])
          try:
            y1.remove([start_row,start_col-1])
            y1.remove([start_row,end_col])
          except ValueError:
            pass
        if direction2 in ['up','down']:
          ship_acquir_empty_space.extend([[start_col,start_row-1],[start_col,end_row]])
          try:
            y1.remove([start_col,start_row-1])
            y1.remove([start_col,end_row])
          except ValueError:
            pass
        ship_positions_2.append([start_row, end_row, start_col, end_col])
        for r in range(start_row, end_row):
            for c in range(start_col, end_col):
                if direction2=='left' or direction2=='right':
                  ship_acquir_empty_space.extend([[r-1,c],[r+1,c]])
                  try:
                    y1.remove([r-1,c])
                    y1.remove([r+1,c])
                  except ValueError:
                    pass
                if direction2=='up' or direction2=='down':
                  ship_acquir_empty_space.extend([[r,c-1],[r,c+1]])
                  try:
                    y1.remove([r,c-1])
                    y1.remove([[r,c+1]])
                  except ValueError:
                    pass
                y1.remove([r,c])
                grid2[r][c] = "O"
    return all_valid


def try_to_place_ship_on_grid2(row, col, direction2, length):
    global grid_size

    start_row, end_row, start_col, end_col = row, row + 1, col, col + 1
    if direction2 == "left":
        if col - length < 0:
            return False
        start_col = col - length + 1

    elif direction2 == "right":
        if col + length >= grid_size:
            return False
        end_col = col + length

    elif direction2 == "up":
        if row - length < 0:
            return False
        start_row = row - length + 1

    elif direction2 == "down":
        if row + length >= grid_size:
            return False
        end_row = row + length

    return validate_grid2_and_place_ship(start_row, end_row, start_col, end_col,direction2)


def print_grid2():
    global alphabet

    debug_mode = True

    alphabet = alphabet[0: len(grid2) + 1]

    for row in range(len(grid2)):
        print(alphabet[row], end=") ")
        for col in range(len(grid2[row])):
            if grid2[row][col] == "O":
                if debug_mode:
                    print("O", end=" ")
                else:
                    print(".", end=" ")
            else:
                print(grid2[row][col], end=" ")
        print("")

    print("  ", end=" ")
    for i in range(len(grid2[0])):
        print(str(i), end=" ")
    print("")


def second_player_grid():
    global grid2
    global grid_size
    global num_of_ships
    global ship_positions_2
    global y1

    random.seed(time.time())

    rows, cols = (grid_size, grid_size)

    grid2 = []
    for r in range(rows):
        row = []
        for c in range(cols):
            row.append(".")
        grid2.append(row)
    num_of_ships_placed = 0

    ship_positions_2 = []
    
    while num_of_ships_placed != num_of_ships:
        ship_size = ship_size_list[0]
        k = random.choice(y1)
        random_row = k[0]
        random_col = k[1]
        direction2 = random.choice(["left", "right", "up", "down"])
        if try_to_place_ship_on_grid2(random_row, random_col, direction2, ship_size):
            ship_size_list.pop(0)
            num_of_ships_placed += 1
    print_grid2()


def check_for_ship_sunk_2(row, col):
    global ship_positions
    global grid2

    for position in ship_positions_2:
        start_row = position[0]
        end_row = position[1]
        start_col = position[2]
        end_col = position[3]
        if start_row <= row <= end_row and start_col <= col <= end_col:
            # Ship found, now check if its all sunk
            for r in range(start_row, end_row):
                for c in range(start_col, end_col):
                    if grid2[r][c] != "X":
                        return False
    return True


def player2_shoot_bullet():
  global grid1
  global num_of_ships_sunk_1
  global bullets_left
  global row
  global col
  global direction
  global hit_count
  global start_row
  global start_col
  global hitting_axies
  global temp1

  row1 = row 
  col1 = col
  temp = True
  
  while temp:

    if hit_count==0:
        x = random.choice(y)
        row = x[0]
        col = x[1]
    if hit_count>=1:
      if hit_count==1:
        direction1 = ["left", "right", "up", "down"]
        if direction!='':
          try:
            direction1.remove(direction)
          except ValueError:
            pass
        if col==0:
          try:
            direction1.remove('left')
          except ValueError:
            pass
        if col==9:
          try:
            direction1.remove('right')
          except ValueError:
            pass
        if row ==0:
          try:
            direction1.remove('up')
          except ValueError:
            pass
        if row ==9:
          try:
            direction1.remove('down')
          except ValueError:
            pass
        direction = random.choice(direction1)
        print(direction)
  
      if direction == "left":
          col = col -1
      if direction == "right":
          col = col + 1
      if direction == "up":
          row = row - 1
      if direction == "down":
          row = row +1
    
      if (row ==10 or row==-1 or col==-1 or col==10 or temp1==1):
        row = start_row
        col = start_col
        temp1=0
        if direction == 'right':
          direction = 'left'
          col = col -1
        elif direction == 'left':
          direction = 'right'
          col = col +1
        elif direction == 'up':
          direction = 'down'
          row = row +1
        elif direction == 'down':
          direction = 'up'
          row = row -1    
    if not([row,col] in hitting_axies):
      hitting_axies.append([row,col])
      temp = False
      temp1=0
    else:
      temp1=1 
  y.remove([row,col])

  if grid1[row][col] == ".":
      print("computer was missed, no ship was shot")
      grid1[row][col] = "#"
      if hit_count>=2:
        temp1 = 1
      if hit_count==1:
        row = row1
        col = col1
  elif grid1[row][col] == "O":
      print("computer  hit!", end=" ")
      grid1[row][col] = "X"
      if check_for_ship_sunk_1(row, col):
          print("A ship was completely sunk!")
          direction = ''
          hit_count = 0
          num_of_ships_sunk_1 += 1
          temp1 = 0
      else:
          if hit_count ==0:
            start_col = col
            start_row = row
            print(col,row)
          hit_count +=1
          print("A ship was shot")
  print_grid1()
  bullets_left -= 1


def check_for_game_over():
    global num_of_ships_sunk_1
    global num_of_ships_sunk_2
    global num_of_ships
    global bullets_left
    global game_over

    if num_of_ships == num_of_ships_sunk_1:
        print("Congrats player 2 win")
        game_over = True
    if num_of_ships == num_of_ships_sunk_2:
        print("congrats player 1 win")
        game_over = True
    elif bullets_left <= 0:
        print("Sorry, no one win You ran out of bullets, try again next time!")
        game_over = True

def main():
    global game_over
    print("-----Welcome to Battleships-----")
    print('player 1 grid for ship positing')
    first_player_grid()
    print("")
    print('player 2 grid for ship positing')
    second_player_grid()

    while game_over is False:
        print("Number of ships remaining for player 1 : " + str(num_of_ships - num_of_ships_sunk_1))
        print("Number of ships remaining for player 2 : "+str(num_of_ships - num_of_ships_sunk_2))
        print("Number of bullets left: " + str(bullets_left))
        #player1_shoot_bullet()
        player2_shoot_bullet()

        print("----------------------------")
        print("")
        check_for_game_over()
if __name__ == '__main__':
    main()

-----Welcome to Battleships-----
player 1 grid for ship positing
A) . . . . . . . . . . 
B) . . . . . . . . . . 
C) . . . . . . . . . . 
D) . . . . . . . . . . 
E) . . . . . . . . . . 
F) . . . . . . . . . . 
G) . . . . . . . . . . 
H) . . . . . . . . . . 
I) . . . . . . . . . . 
J) . . . . . . . . . . 
   0 1 2 3 4 5 6 7 8 9 
enter_ship_size : 3
enter_row : 0
enter_col : 0
enter_direction_from_["left", "right", "up", "down"] : down
A) O . . . . . . . . . 
B) O . . . . . . . . . 
C) O . . . . . . . . . 
D) . . . . . . . . . . 
E) . . . . . . . . . . 
F) . . . . . . . . . . 
G) . . . . . . . . . . 
H) . . . . . . . . . . 
I) . . . . . . . . . . 
J) . . . . . . . . . . 
   0 1 2 3 4 5 6 7 8 9 
enter_ship_size : 9
enter_row : 0
enter_col : 9
enter_direction_from_["left", "right", "up", "down"] : down
A) O . . . . . . . . O 
B) O . . . . . . . . O 
C) O . . . . . . . . O 
D) . . . . . . . . . O 
E) . . . . . . . . . O 
F) . . . . . . . . . O 
G) . . . . . . . . . O 
H) . . . . . . . . . O 